# OLD NOTEBOOK - EXAMPLE OF PULLING BPS DATA FROM PREVIOUS PROJECT (pulls permits for counties in major U.S. Metros)

Author: Dana Chermesh, Regional Planning intern; NYC DCP<br>
Summer 2018

### _US Metros comparison  Notebook no.5_ 
# Housing Permitted 2000-2008; 2008-2017
retrieved from [Census Bureau Building Permits Survey](https://www.census.gov/construction/bps/), County and Place level

### _Approach:_
- Analyze permits issuance pre- and post-recession, meaning for 2000-2008 and for 2008-2017.
----


In [122]:
import pandas as pd

# reading in my api key saved in censusAPI.py as
# myAPI = 'XXXXXXXXXXXXXXX'
# request an api key in: https://api.census.gov/data/key_signup.html
from censusAPI import myAPI

# Data

## 1.1 Total housing units 2000 (base point); _Decennial Census 2000, SF1_
Data were retrieved using my census API (_**see explanation in README.md of this repo or in former notebooks**_)
- [variables](https://api.census.gov/data/2000/sf1/variables.html)

In [123]:
# total POP and total housing units for all counties in the US, 2010
# P001001 = total pop
# H001001 = total housing units

housing00 = pd.read_json('https://api.census.gov/data/2000/sf1?get=P001001,H001001'+
                         '&for=county:*&in=state:*&key='+myAPI)
housing00.columns = housing00.iloc[0]
housing00 = housing00[1:]

housing00.columns = ['pop2000', 'hu2000', 'state', 'county']
housing00['STCO'] = housing00[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

print(housing00.shape)
housing00.head()

(3141, 5)


pop2000 hu2000 state county   STCO
1   43671  17662    01    001  01001
2  140415  74285    01    003  01003
3   29038  12461    01    005  01005
4   20826   8345    01    007  01007
5   51024  21158    01    009  01009

## Total housing units 2008 (base point); _PEP housing 2000-2008_ 
** Data need to be downloaded to your local machine, then read in 2008 estimates only 
- Source: [Housing Units Intercensal Datasets: 2000-2010](https://www.census.gov/data/datasets/time-series/demo/popest/intercensal-2000-2010-housing-units.html)
- [Direct download](https://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/housing/hu-est00int-tot.csv)

In [124]:
housing08 = pd.read_excel('../rp-USmetros_comparison/data/hu-est00int-tot.xlsx')
housing08 = housing08[housing08['SUMLEV'] != 40]
housing08 = housing08.drop(['HUESTBASE2000', 'HUEST_2000', 'HUEST_2001', 'HUEST_2002',
                            'HUEST_2003', 'HUEST_2004', 'HUEST_2005', 'HUEST_2006',
                            'HUEST_2007', 'HUEST_2009', 'HUCENSUS2010', 'HUEST_2010',
                            'SUMLEV'], axis=1)

housing08.columns = ['state', 'county', 'NAME', 'hu2008']
housing08['state'] = housing08['state'].apply(lambda x: '{0:0>2}'.format(x))
housing08['county'] = housing08['county'].astype(int).apply(lambda x: '{0:0>3}'.format(x))
housing08['STCO'] = housing08[['state', 'county']].apply(lambda x: ''.join(x), axis=1)

housing08 = housing08.drop(['state', 'county'], axis=1)

print(housing08.shape)
housing08.head()

(3143, 3)


NAME  hu2008   STCO
1  Autauga County   21605  01001
2  Baldwin County  102011  01003
3  Barbour County   12009  01005
4     Bibb County    8863  01007
5   Blount County   23500  01009

### Places

----

# 1.2 Perimts issuance data from 2008 to 2017 - [Building Permits Survey](https://www.census.gov/construction/bps/)
The data were retrieved from the Census Bureau [Building Permits Survey](https://www.census.gov/construction/bps/), [Permits by County or Place](http://www2.census.gov/econ/bps).

For downloading the data, please go to the [County/](https://www2.census.gov/econ/bps/County/) page or the [Places/](https://www2.census.gov/econ/bps/Place/) page, and choose:
- [co2017a.txt](https://www2.census.gov/econ/bps/County/co2017a.txt) 
- [co2016a.txt](https://www2.census.gov/econ/bps/County/co2016a.txt) 
- [co2015a.txt](https://www2.census.gov/econ/bps/County/co2015a.txt) 
- [co2014a.txt](https://www2.census.gov/econ/bps/County/co2014a.txt) 
- [co2013a.txt](https://www2.census.gov/econ/bps/County/co2013a.txt) 
- [co2012a.txt](https://www2.census.gov/econ/bps/County/co2012a.txt) 
- [co2011a.txt](https://www2.census.gov/econ/bps/County/co2011a.txt) 
- [co2010a.txt](https://www2.census.gov/econ/bps/County/co2010a.txt) 

and:
- [ne2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [mw2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [so2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)
- [we2017a.txt](https://www2.census.gov/econ/bps/Place/Northeast%20Region/ne2017a.txt)<br>
.<br>
.<br>
.<br>
.<br>

Data can be read directly to this notebook using pandas `read_table`, as bellow.

## 2008-2017 _(including 2008)_ Permits 

In [125]:
tablesCO = ['co2017a', 'co2016a', 'co2015a', 'co2014a',
            'co2013a', 'co2012a', 'co2011a', 'co2010a',
            'co2009a', 'co2008a']

COdata08 = []

for year in tablesCO:
    
    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units'] = df['1unit'] + df['2unit']
    df['+3units'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata08.append(df)

COall08_17 = pd.concat(COdata08).groupby('STCO').sum()
print(COall08_17.shape)
print(COall08_17.dtypes)
COall08_17.head()

data: co2017a (3038, 5)
data: co2016a (3039, 5)
data: co2015a (3044, 5)
data: co2014a (3038, 5)
data: co2013a (3027, 5)
data: co2012a (3026, 5)
data: co2011a (3026, 5)
data: co2010a (3026, 5)
data: co2009a (3026, 5)
data: co2008a (3026, 5)
(3061, 2)
1-2units    int64
+3units     int64
dtype: object


1-2units  +3units
STCO                    
01001      1419      312
01003     13407     2229
01005        81       40
01007       140       42
01009       107       40

## 2000-2008 _(including 2000, excluding 2008)_ Permits

In [126]:
tablesCO00 = ['co2007a', 'co2006a', 'co2005a', 'co2004a',
              'co2003a', 'co2002a', 'co2001a', 'co2000a']

COdata00 = []

for year in tablesCO00:

    df = pd.read_table('https://www2.census.gov/econ/bps/County/'+year+'.txt', 
            header=0, sep=r'\,|\t', engine='python').iloc[:,:18]

    df.columns = df.iloc[0]
    df = df[1:].set_index(['Name'])

    df = df.drop(['Code','Bldgs', 'Value'], axis=1)
    df.columns = ['State', 'County', '1unit', '2unit', '3-4unit', '+5unit']

    df = df.astype(int)
    df['1-2units_00'] = df['1unit'] + df['2unit']
    df['+3units_00'] = df['3-4unit'] + df['+5unit']
    df = df.drop(['1unit', '2unit', '3-4unit', '+5unit'], axis=1)

    df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
    df['County'] = df['County'].apply(lambda x: '{0:0>3}'.format(x))

    df['STCO'] = df[['State', 'County']].apply(lambda x: ''.join(x), axis=1)
    print('data:',year ,df.shape)
    COdata00.append(df)

COall00_08 = pd.concat(COdata00).groupby('STCO').sum()
print(COall00_08.shape)
print(COall00_08.dtypes)
COall00_08.head()

data: co2007a (3026, 5)
data: co2006a (3026, 5)
data: co2005a (3027, 5)
data: co2004a (3027, 5)
data: co2003a (3021, 5)
data: co2002a (3013, 5)
data: co2001a (3014, 5)
data: co2000a (3018, 5)
(3050, 2)
1-2units_00    int64
+3units_00     int64
dtype: object


1-2units_00  +3units_00
STCO                          
01001         2203         224
01003        21082        9032
01005          180          40
01007           42           0
01009          389          91

## 1.3 Reading in geo-coded dataset
created on a different notebook, please refer to [notebook no.0: 0-US_Metro_Comparison_Geographies.ipynb](https://github.com/NYCPlanning/rp-USmetros_comparison/blob/master/0-US_Metro_Comparison_Geographies.ipynb)

In [127]:
geo = pd.read_csv('exports/USmetros_COUNTIESraw.csv').iloc[:,1:]            
geo['CSA'] = geo['CSA'].astype(str).str[:-2]
geo['STCO'] = geo['STCO'].astype(str)

print(geo.shape)
print(geo.dtypes)
geo.head()

(275, 3)
STCO        object
NAMELSAD    object
CSA         object
dtype: object


STCO         NAMELSAD  CSA
0  51179  Stafford County  548
1  18073    Jasper County  176
2  13171     Lamar County  122
3  25025   Suffolk County  148
4  48147    Fannin County  206

In [128]:
geo2 = pd.read_csv('data/USmetros_full.csv').iloc[:,:-2] \
            .drop(['Unnamed: 0', 'SHAPE_AREA'], axis=1)
geo2['STCO'] = geo2['STCO'].apply(lambda x: '{0:0>5}'.format(x))
geo2['CSA'] = geo2['CSA'].astype(str)
geo2['STCO'] = geo2['STCO'].astype(str)

print(geo2.shape)
print(geo2.dtypes)
geo2.head()

(270, 4)
CSA            object
CSA_name       object
County_name    object
STCO           object
dtype: object


CSA                            CSA_name   County_name   STCO
0  488  San Jose-San Francisco-Oakland, CA       Alameda  06001
1  488  San Jose-San Francisco-Oakland, CA  Contra Costa  06013
2  488  San Jose-San Francisco-Oakland, CA         Marin  06041
3  488  San Jose-San Francisco-Oakland, CA          Napa  06055
4  488  San Jose-San Francisco-Oakland, CA    San Benito  06069

In [129]:
# Chicago missing county
geo2[geo2['CSA'] == '176'].iloc[:,:2].shape

(18, 2)

In [130]:
# LA missing counties
print(geo[geo['CSA'] == '348'])
print(geo2[geo2['CSA'] == '348'])

     STCO               NAMELSAD  CSA
33   6111         Ventura County  348
35   6037     Los Angeles County  348
97   6071  San Bernardino County  348
151  6065       Riverside County  348
221  6059          Orange County  348
    CSA                    CSA_name  County_name   STCO
12  348  Los Angeles-Long Beach, CA  Los Angeles  06037
13  348  Los Angeles-Long Beach, CA       Orange  06059


### Fixing counties df to include all LA + Chicago counties

In [131]:
# Chicago
geo2 = pd.concat([geo[geo['STCO'] == '17031'],geo2], axis=0, ignore_index=True)
geo2['CSA_name'][geo2['STCO'] == '17031'] = 'Chicago-Naperville, IL-IN-WI'
geo2['County_name'][geo2['STCO'] == '17031'] = 'Cook'

# LA: Ventura County
geo2 = pd.concat([geo[geo['STCO'] == '6111'],geo2], axis=0, ignore_index=True)
geo2['CSA_name'][geo2['STCO'] == '6111'] = 'Los Angeles-Long Beach, CA'
geo2['County_name'][geo2['STCO'] == '6111'] = 'Ventura'
# LA: San Bernardino County
geo2 = pd.concat([geo[geo['STCO'] == '6071'],geo2], axis=0, ignore_index=True)
geo2['CSA_name'][geo2['STCO'] == '6071'] = 'Los Angeles-Long Beach, CA'
geo2['County_name'][geo2['STCO'] == '6071'] = 'San Bernardino'
# LA: Riverside County
geo2 = pd.concat([geo[geo['STCO'] == '6065'],geo2], axis=0, ignore_index=True)
geo2['CSA_name'][geo2['STCO'] == '6065'] = 'Los Angeles-Long Beach, CA'
geo2['County_name'][geo2['STCO'] == '6065'] = 'Riverside'

geo2['STCO'] = geo2['STCO'].apply(lambda x: '{0:0>5}'.format(x))
geo2 = geo2.drop(['NAMELSAD'], axis=1)

geo = geo2

print('Chicago shape: ', geo[geo['CSA'] == '176'].shape)
print('LA shape: ', geo[geo['CSA'] == '348'].shape)
print(geo.shape)
geo[geo['CSA'] == '348'].head()

Chicago shape:  (19, 4)
LA shape:  (5, 4)
(274, 4)


/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is

CSA                    CSA_name     County_name   STCO
0   348  Los Angeles-Long Beach, CA       Riverside  06065
1   348  Los Angeles-Long Beach, CA  San Bernardino  06071
2   348  Los Angeles-Long Beach, CA         Ventura  06111
16  348  Los Angeles-Long Beach, CA     Los Angeles  06037
17  348  Los Angeles-Long Beach, CA          Orange  06059

In [141]:
for i in geo['CSA'].unique():
    print('No. of counties in CSA {}: {}'\
          .format(i, geo[geo['CSA']==i].shape[0]))
    
print(geo['CSA'].unique())

No. of counties in CSA 348: 5
No. of counties in CSA 176: 19
No. of counties in CSA 488: 12
No. of counties in CSA 216: 12
No. of counties in CSA 408: 31
No. of counties in CSA 148: 19
No. of counties in CSA 428: 16
No. of counties in CSA 548: 40
No. of counties in CSA 370: 7
No. of counties in CSA 122: 39
No. of counties in CSA 220: 10
No. of counties in CSA 378: 21
No. of counties in CSA 206: 20
No. of counties in CSA 288: 14
No. of counties in CSA 500: 9
['348' '176' '488' '216' '408' '148' '428' '548' '370' '122' '220' '378'
 '206' '288' '500']


In [138]:
geo[geo['CSA'] == '216'].iloc[:,-2:]

County_name   STCO
18        Adams  08001
19     Arapahoe  08005
20      Boulder  08013
21   Broomfield  08014
22  Clear Creek  08019
23       Denver  08031
24      Douglas  08035
25       Elbert  08039
26       Gilpin  08047
27    Jefferson  08059
28         Park  08093
29         Weld  08123

In [132]:
geo.to_csv('data/USmetros_full_correct.csv')

## 1.4 Merging datasets

### merging 00-08 with 08-17 datasets

In [133]:
permits = COall00_08.merge(COall08_17, on='STCO')

print(permits.shape)
permits.head()

(3029, 4)


1-2units_00  +3units_00  1-2units  +3units
STCO                                             
01001         2203         224      1419      312
01003        21082        9032     13407     2229
01005          180          40        81       40
01007           42           0       140       42
01009          389          91       107       40

In [113]:
type(permits.index[0])

str

### merging permits with geo dataset

In [114]:
BPcounties = permits.merge(geo, left_index=True, right_on='STCO').set_index('County_name')

print(BPcounties.shape)
BPcounties.head()

(274, 7)


1-2units_00  +3units_00  1-2units  +3units  CSA  \
County_name                                                     
Alameda             16510       17985     14921    21559  488   
Contra Costa        34464        8556     13905     5476  488   
Los Angeles         81456       85636     41799   101532  348   
Marin                3384        1550      1409     1260  488   
Napa                 4360        1144      1378      288  488   

                                        CSA_name   STCO  
County_name                                              
Alameda       San Jose-San Francisco-Oakland, CA  06001  
Contra Costa  San Jose-San Francisco-Oakland, CA  06013  
Los Angeles           Los Angeles-Long Beach, CA  06037  
Marin         San Jose-San Francisco-Oakland, CA  06041  
Napa          San Jose-San Francisco-Oakland, CA  06055

In [115]:
print(BPcounties[BPcounties['CSA']=='348'].shape)
BPcounties[BPcounties['CSA']=='348']

(5, 7)


1-2units_00  +3units_00  1-2units  +3units  CSA  \
County_name                                                       
Los Angeles           81456       85636     41799   101532  348   
Orange                41808       32441     30152    40311  348   
Riverside            167696       23867     43901    11637  348   
San Bernardino        80855       11867     22027     9774  348   
Ventura               17637        6835      4769     6122  348   

                                  CSA_name   STCO  
County_name                                        
Los Angeles     Los Angeles-Long Beach, CA  06037  
Orange          Los Angeles-Long Beach, CA  06059  
Riverside       Los Angeles-Long Beach, CA  06065  
San Bernardino  Los Angeles-Long Beach, CA  06071  
Ventura         Los Angeles-Long Beach, CA  06111

### Exporting all counties housing permits 2017 data to .csv

In [116]:
BPcounties.to_csv('exports/BPcounties00-08-17.csv')

## groupby CSAs to sum housing permits by Metro, merging 2010 Census Data of total housing units and total pop 

In [117]:
housing00['pop2000'] = housing00['pop2000'].astype(int)
housing00['hu2000'] = housing00['hu2000'].astype(int)

housing00.dtypes

pop2000     int64
hu2000      int64
state      object
county     object
STCO       object
dtype: object

In [118]:
housing08['hu2008'] = housing08['hu2008'].astype(int)

housing08.dtypes

NAME      object
hu2008     int64
STCO      object
dtype: object

In [119]:
hpCSA00_08_17 = housing00.merge(housing08, on='STCO')
hpCSA00_08_17 = hpCSA00_08_17.merge(BPcounties, on='STCO').groupby(['CSA', 'CSA_name']).sum()

print(hpCSA00_08_17.shape)
hpCSA00_08_17

(15, 7)


pop2000   hu2000  \
CSA CSA_name                                                              
122 Atlanta--Athens-Clarke County--Sandy Springs, GA   4778990  1853158   
148 Boston-Worcester-Providence, MA-RI-NH-CT           7630016  3135867   
176 Chicago-Naperville, IL-IN-WI                       9465353  3613085   
206 Dallas-Fort Worth, TX-OK                           5565005  2180002   
216 Denver-Aurora, CO                                  2629980  1068646   
220 Detroit-Warren-Ann Arbor, MI                       5456428  2208124   
288 Houston-The Woodlands, TX                          4878216  1865805   
348 Los Angeles-Long Beach, CA                        16373645  5678148   
370 Miami-Fort Lauderdale-Port St. Lucie, FL           5475847  2379888   
378 Minneapolis-St. Paul, MN-WI                        3335000  1312902   
408 New York-Newark, NY-NJ-CT-PA                      21491898  8341615   
428 Philadelphia-Reading-Camden, PA-NJ-DE-MD           6688798  2740528   
488 San Jose-San Francisco-Oakland, CA                 7656194  2856934   
500 Seattle-Tacoma, WA                                 3775744  1564957   
548 Washington-Baltimore-Arlington, DC-MD-VA-WV-PA     7981257  3204298   

                                                       hu2008  1-2units_00  \
CSA CSA_name                                                                 
122 Atlanta--Athens-Clarke County--Sandy Springs, GA  2408557       454314   
148 Boston-Worcester-Providence, MA-RI-NH-CT          3357101       148231   
176 Chicago-Naperville, IL-IN-WI                      3950014       268563   
206 Dallas-Fort Worth, TX-OK                          2648141       328860   
216 Denver-Aurora, CO                                 1266734       153404   
220 Detroit-Warren-Ann Arbor, MI                      2339936       140823   
288 Houston-The Woodlands, TX                         2316282       330860   
348 Los Angeles-Long Beach, CA                        6243248       389452   
370 Miami-Fort Lauderdale-Port St. Lucie, FL          2761106       224837   
378 Minneapolis-St. Paul, MN-WI                       1516804       152957   
408 New York-Newark, NY-NJ-CT-PA                      8819237       270955   
428 Philadelphia-Reading-Camden, PA-NJ-DE-MD          2929769       166248   
488 San Jose-San Francisco-Oakland, CA                3122754       155975   
500 Seattle-Tacoma, WA                                1811275       163777   
548 Washington-Baltimore-Arlington, DC-MD-VA-WV-PA    3631935       315691   

                                                      +3units_00  1-2units  \
CSA CSA_name                                                                 
122 Atlanta--Athens-Clarke County--Sandy Springs, GA      123679    156366   
148 Boston-Worcester-Providence, MA-RI-NH-CT               54704     85160   
176 Chicago-Naperville, IL-IN-WI                          112777     70136   
206 Dallas-Fort Worth, TX-OK                               92292    223385   
216 Denver-Aurora, CO                                      55947     85517   
220 Detroit-Warren-Ann Arbor, MI                           28409     49786   
288 Houston-The Woodlands, TX                             103779    310443   
348 Los Angeles-Long Beach, CA                            160646    142648   
370 Miami-Fort Lauderdale-Port St. Lucie, FL              142164     68614   
378 Minneapolis-St. Paul, MN-WI                            45713     65048   
408 New York-Newark, NY-NJ-CT-PA                          239743    129452   
428 Philadelphia-Reading-Camden, PA-NJ-DE-MD               41112     86344   
488 San Jose-San Francisco-Oakland, CA                     87486     76288   
500 Seattle-Tacoma, WA                                     68909    107305   
548 Washington-Baltimore-Arlington, DC-MD-VA-WV-PA         97779    182690   

                                                      +3units  
CSA CSA_name                                                 

### Exporting CSAs housing permits data to .csv

In [120]:
hpCSA00_08_17.to_csv('exports/BP00-08-17CSA.csv')

-----

# PLACES 
Major Cities within the Regions

### _Note#1: PLACES in the housing permits survey are separated to Midwest, Northeast, South and West Regions; data were downloaded through each of these for years 2010 - 2017 and were concatenated, then cleaned to include our targeted cities only_

### _Note#2: NYC is considered as 5 places, which are its 5 boroughs_

# _** There were changes in GEOID's for several places_

----

## Reading in geo-coded places dataset

Created by Dara Goldberg, DCP Regional Planning

In [13]:
geoPlace = pd.read_csv('../rp-USmetros_comparison/data/Geocoded_places.csv')
geoPlace['GEOID'] = geoPlace['GEOID'].apply(lambda x: '{0:0>7}'.format(x))
geoPlace['GEOID'] = geoPlace['GEOID'].astype(int)

print(geoPlace.shape)
print(geoPlace.dtypes)
geoPlace.head(3)

(19, 5)
GEOID         int64
NAMELSAD     object
NAME         object
CSA           int64
ALAND_mi    float64
dtype: object


GEOID                        NAMELSAD           NAME  CSA   ALAND_mi
0  644000    Los Angeles city, California    Los Angeles  348  468.65867
1  653000        Oakland city, California        Oakland  488   55.89604
2  667000  San Francisco city, California  San Francisco  488   46.90564

## BPS data acquisition for places
## 2008-2017 _(including 2008)_ Permits 

In [22]:
# all needed files to be read in, aggregated by area
Northeast = ['ne2017a', 'ne2016a', 'ne2015a', 'ne2014a','ne2013a', 
             'ne2012a', 'ne2011a', 'ne2010a', 'ne2009a', 'ne2008a']
Midwest = ['mw2017a', 'mw2016a', 'mw2015a', 'mw2014a','mw2013a', 
           'mw2012a', 'mw2011a', 'mw2010a', 'mw2009a', 'mw2008a']
South = ['so2017a', 'so2016a', 'so2015a', 'so2014a','so2013a', 
         'so2012a', 'so2011a', 'so2010a', 'so2009a', 'so2008a']
West = ['we2017a', 'we2016a', 'we2015a', 'we2014a','we2013a', 
        'we2012a', 'we2011a', 'we2010a', 'we2009a', 'we2008a']

# all US areas
US_AREAS = {'Northeast' : Northeast,
            'Midwest' : Midwest,
            'South' : South,
            'West' : West}

# empty list for all dataframes to be added
USA = []

# reading data into python pandas
for area in US_AREAS:
    for year in US_AREAS[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Pop', 'Footnote', 'Census Place',
                      '6-Digit', 'FIPS MCD', 'Number of',
                      'Zip', 'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units'] = df['1unit'] + df['2unit']
        df['+3units'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['Place'] = df['Place'].apply(lambda x: '{0:0>5}'.format(x))
        df['GEOID'] = df[['State', 'Place']].apply(lambda x: ''.join(x), axis=1)

        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', 'Place'], axis=1)

        df['1-2units'] = df['1-2units'].astype(int)
        df['+3units'] = df['+3units'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA.append(df)


hp_Places = pd.concat(USA).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
hp_Places = hp_Places[11:]

print(hp_Places.shape)
print(hp_Places.dtypes)
hp_Places.head(15)

mw2017a
mw2016a
mw2015a
mw2014a
mw2013a
mw2012a
mw2011a
mw2010a
mw2009a
mw2008a
we2017a
we2016a
we2015a
we2014a
we2013a
we2012a
we2011a
we2010a
we2009a
we2008a
so2017a
so2016a
so2015a
so2014a
so2013a
so2012a
so2011a
so2010a
so2009a
so2008a
ne2017a
ne2016a
ne2015a
ne2014a
ne2013a
ne2012a
ne2011a
ne2010a
ne2009a
ne2008a
(30152, 5)
GEOID       object
CSA         object
Name        object
1-2units     int64
+3units      int64
dtype: object


GEOID  CSA            Name  1-2units  +3units
11  0100124   222       Abbeville         5        0
12  0100460   142      Adamsville        10        0
13  0100676   999      Akron town         0        0
14  0100820   142       Alabaster       877        0
15  0100988   999     Albertville       100        8
16  0101132   999  Alexander City       130       56
17  0101228   999      Aliceville        15        0
18  0101708   999       Andalusia        81        0
19  0101852   999        Anniston        26        0
20  0102116   999            Arab       122        0
21  0102260   290    Ardmore town        61        0
22  0102836   222    Ashford town        58        0
23  0102860   999         Ashland         1        0
24  0102860   999    Ashland town         6        0
25  0102908   142        Ashville        38        0

In [78]:
hp_Places[hp_Places['Name'] == 'Chicago']

GEOID  CSA     Name  1-2units  +3units
4099  1714000  176  Chicago      4442    43092

## 2000-2008 _(including 2000, excluding 2008)_ Permits 
### ** _at 2007 the data were changes as two columns were added; the data for 2007 and for 2000-2006 (included) were obtained separately_

In [68]:
# all needed files to be read in, aggregated by area
Northeast07 = ['ne2007a']
Midwest07 = ['mw2007a']
South07 = ['so2007a']
West07 = ['we2007a']

# all US areas
US_AREAS07 = {'Northeast' : Northeast07,
            'Midwest' : Midwest07,
            'South' : South07,
            'West' : West07}

# empty list for all dataframes to be added
USA00 = []

# reading data into python pandas
for area in US_AREAS07:
    for year in US_AREAS07[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Census Place',
                      'Place','FIPS MCD', 'Pop', 'CSA', 'CBSA',
                      'Footnote', 'Central', 'Zip','Region', 'Division', 
                      'Number of','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Pop', 'Footnote', 'Census Place',
                      'Place', 'FIPS MCD', 'Number of',
                      'Zip', 'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units00'] = df['1unit'] + df['2unit']
        df['+3units00'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['6-Digit'] = df['6-Digit'].apply(lambda x: '{0:0>6}'.format(x))
        df['GEOID'] = df[['State', '6-Digit']].apply(lambda x: ''.join(x), axis=1)

        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', '6-Digit'], axis=1)

        df['1-2units00'] = df['1-2units00'].astype(int)
        df['+3units00'] = df['+3units00'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA00.append(df)


# hp_Places07 = pd.concat(USA07).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
# hp_Places07 = hp_Places07[11:]

# print(hp_Places07.shape)
# print(hp_Places07.dtypes)
# hp_Places07.head(15)

mw2007a
we2007a
so2007a
ne2007a


In [69]:
# all needed files to be read in, aggregated by area
Northeast00 = ['ne2006a', 'ne2005a', 'ne2004a', 
               'ne2003a', 'ne2002a', 'ne2001a', 'ne2000a']
Midwest00 = ['mw2006a', 'mw2005a', 'mw2004a', 
             'mw2003a', 'mw2002a', 'mw2001a', 'mw2000a']
South00 = ['so2006a', 'so2005a', 'so2004a', 
           'so2003a', 'so2002a', 'so2001a', 'so2000a']
West00 = ['we2006a', 'we2005a', 'we2004a',
          'we2003a', 'we2002a', 'we2001a', 'we2000a']

# all US areas
US_AREAS00 = {'Northeast' : Northeast00,
            'Midwest' : Midwest00,
            'South' : South00,
            'West' : West00}

# empty list for all dataframes to be added
USA00 = []

# reading data into python pandas
for area in US_AREAS00:
    for year in US_AREAS00[area]:
        df = year
        print(df)
        df = pd.read_table('https://www2.census.gov/econ/bps/Place/'+
                             area+'%20Region/'+
                             year+'.txt', 
                             header=0, sep=r'\,|\t', engine='python').iloc[:,:28]

        df.columns = df.iloc[0]
        df = df[1:].set_index(['Name'])

        df = df.drop(['Bldgs', 'Value'], axis=1)
        df.columns = ['State', '6-Digit', 'County', 'Place', 
                      'CSA', 'CBSA', 'Footnote', 'Central', 
                      'Zip','Region', 'Division', 'Number of',
                      'Place2','1unit', '2unit', '3-4unit', '+5unit']
        df = df.drop(['Central', 'Footnote', 'Place2',
                      'Place', 'Number of', 'Zip',
                      'Region', 'Division'], axis=1)

        df['1unit'] = df['1unit'].astype(int)
        df['2unit'] = df['2unit'].astype(int)
        df['3-4unit'] = df['3-4unit'].astype(int)
        df['+5unit'] = df['+5unit'].astype(int)

        df['1-2units00'] = df['1unit'] + df['2unit']
        df['+3units00'] = df['3-4unit'] + df['+5unit']

        # creating 'GEOID' column from state and place
        df['State'] = df['State'].apply(lambda x: '{0:0>2}'.format(x))
        df['6-Digit'] = df['6-Digit'].apply(lambda x: '{0:0>6}'.format(x))
        df['GEOID'] = df[['State', '6-Digit']].apply(lambda x: ''.join(x), axis=1)
        
        # Dropping columns
        df = df.drop(['1unit', '2unit', '3-4unit', '+5unit',
                                          'State', '6-Digit'], axis=1)

        df['1-2units00'] = df['1-2units00'].astype(int)
        df['+3units00'] = df['+3units00'].astype(int)
        df['GEOID'] = df['GEOID'].replace(' ', '')
        for index, row in df.iterrows():
            if row['GEOID'] == '2500025':
                row['GEOID'] = '2507000'
        USA00.append(df)


hp_Places00 = pd.concat(USA00).groupby(['GEOID', 'CSA', 'Name']).sum().reset_index()
hp_Places00 = hp_Places00[11:]

print(hp_Places00.shape)
print(hp_Places00.dtypes)
hp_Places00.head(15)

mw2006a
mw2005a
mw2004a
mw2003a
mw2002a
mw2001a
mw2000a
we2006a
we2005a
we2004a
we2003a
we2002a
we2001a
we2000a
so2006a
so2005a
so2004a
so2003a
so2002a
so2001a
so2000a
ne2006a
ne2005a
ne2004a
ne2003a
ne2002a
ne2001a
ne2000a
(40940, 5)
GEOID         object
CSA           object
Name          object
1-2units00     int64
+3units00      int64
dtype: object


GEOID   CSA          Name  1-2units00  +3units00
11  01008000   999    Aliceville           0          0
12  01008000  9999    Aliceville           0          3
13  01012000   999     Andalusia           0         82
14  01012000  9999     Andalusia           0         26
15  01014000  0450      Anniston           0          0
16  01014000   999      Anniston           0         43
17  01016000   999          Arab           4        109
18  01016000  9999          Arab           0         81
19  01017000   290  Ardmore town           0         29
20  01017000  3440  Ardmore town           0          6
21  01020000   218   Ariton town           0          0
22  01020000  2180   Ariton town           0          0
23  01020000   999   Ariton town           0          0
24  01022000  2180       Ashford           0         12
25  01022000   222       Ashford           0         19

## 2.2 Merging datasets
### Merging 00-08 with 08-17 datasets

In [72]:
# converting datatype to int
hp_Places['GEOID'] = hp_Places['GEOID'].apply(lambda x: '{0:0>8}'.format(x))
hp_Places['GEOID'] = hp_Places['GEOID'].astype(int)
hp_Places['CSA'] = hp_Places['CSA'].astype(int)

hp_Places00['GEOID'] = hp_Places00['GEOID'].astype(int)
hp_Places00['CSA'] = hp_Places00['CSA'].astype(int)

In [73]:
permits00 = hp_Places00.merge(hp_Places, on='GEOID')

print(permits00.shape)
permits00.head()

(318, 9)


GEOID  CSA_x       Name_x  1-2units00  +3units00  CSA_y           Name_y  \
0  1213000   2180  Gordon town           0          4    999        Clewiston   
1  1213000    222  Gordon town           0          0    999        Clewiston   
2  1224000    516  Gulf Shores          34        657    370  Fort Lauderdale   
3  1224000    516  Gulf Shores          34        657    999  Fort Lauderdale   
4  1224000   5160  Gulf Shores          36        673    370  Fort Lauderdale   

   1-2units  +3units  
0       104       53  
1       104       53  
2       500     1409  
3       409     1244  
4       500     1409

In [74]:
permits00[permits00['CSA_y'] == 408]

Empty DataFrame
Columns: [GEOID, CSA_x, Name_x, 1-2units00, +3units00, CSA_y, Name_y, 1-2units, +3units]
Index: []

In [ ]:
# BPcounties = permits.merge(geo, left_index=True, right_on='STCO').set_index('County_name')

# print(BPcounties.shape)
# BPcounties.head()

### Merging all places with geo-coded places dataset

In [77]:
# merging all places with our target places list
BPS_place = geoPlace.merge(hp_Places, on = 'GEOID')

BPS_place.rename(columns={'CSA_x':'CSA'}, inplace=True)
BPS_place = BPS_place.drop(['CSA_y', 'ALAND_mi', 'NAME'], axis=1)
BPS_place['GEOID'] = BPS_place['GEOID'].apply(lambda x: '{0:0>7}'.format(x))
BPS_place['CSA'] = BPS_place['CSA'].astype(str)
BPS_place = BPS_place.drop(BPS_place.index[7]).reset_index().drop(['index'], axis=1)

print(BPS_place.shape)
BPS_place

(23, 6)


GEOID                               NAMELSAD  CSA  \
0   0644000           Los Angeles city, California  348   
1   0653000               Oakland city, California  488   
2   0667000         San Francisco city, California  488   
3   0668000              San Jose city, California  488   
4   0820000                  Denver city, Colorado  216   
5   1150000  Washington city, District of Columbia  548   
6   1245000                    Miami city, Florida  370   
7   1304000                  Atlanta city, Georgia  122   
8   1714000                 Chicago city, Illinois  176   
9   2507000             Boston city, Massachusetts  148   
10  2622000                 Detroit city, Michigan  220   
11  2743000            Minneapolis city, Minnesota  378   
12  2758000               St. Paul city, Minnesota  378   
13  3651000                New York city, New York  408   
14  3651000                New York city, New York  408   
15  3651000                New York city, New York  408   
16  3651000                New York city, New York  408   
17  3651000                New York city, New York  408   
18  4260000        Philadelphia city, Pennsylvania  428   
19  4819000                     Dallas city, Texas  206   
20  4827000                 Fort Worth city, Texas  206   
21  4835000                    Houston city, Texas  288   
22  5363000               Seattle city, Washington  500   

                     Name  1-2units  +3units  
0             Los Angeles     16413    73435  
1                 Oakland      1061     8600  
2           San Francisco       890    26362  
3                San Jose      1862    22477  
4                  Denver     14679    36980  
5              Washington      3196    30767  
6                   Miami       641    18306  
7                 Atlanta      4965    31340  
8                 Chicago      4442    43092  
9                  Boston      1097    21450  
10                Detroit       704     2878  
11            Minneapolis       987    16502  
12               St. Paul       452     3500  
13          Bronx borough      1004    26466  
14       Brooklyn borough      1815    69250  
15      Manhattan borough        72    47866  
16         Queens borough      5263    39590  
17  Staten Island borough      5255     2371  
18           Philadelphia      7915    16462  
19                 Dallas     12489    59802  
20             Fort Worth     33656    22621  
21                Houston     40686    66918  
22                Seattle      8042    56102

### _Summing NYC five boroughs to one row_

In [17]:
BPS_place = BPS_place.append(BPS_place[13:18].sum(numeric_only=True), ignore_index=True)

BPS_place['1-2units'] = BPS_place['1-2units'].astype(int)
BPS_place['+3units'] = BPS_place['+3units'].astype(int)

# assigning values to NaNs
BPS_place['GEOID'][-1:] = '3651000'
BPS_place['NAMELSAD'][-1:] = 'New York city, New York'
BPS_place['CSA'][-1:] = '408'
BPS_place['Name'][-1:] = 'New York'

BPS_place = BPS_place.drop(BPS_place.index[13:18]).reset_index().drop(['index'], axis=1)

print(BPS_place.shape)
BPS_place

(19, 6)


/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on 

GEOID                               NAMELSAD  CSA           Name  \
0   0644000           Los Angeles city, California  348    Los Angeles   
1   0653000               Oakland city, California  488        Oakland   
2   0667000         San Francisco city, California  488  San Francisco   
3   0668000              San Jose city, California  488       San Jose   
4   0820000                  Denver city, Colorado  216         Denver   
5   1150000  Washington city, District of Columbia  548     Washington   
6   1245000                    Miami city, Florida  370          Miami   
7   1304000                  Atlanta city, Georgia  122        Atlanta   
8   1714000                 Chicago city, Illinois  176        Chicago   
9   2507000             Boston city, Massachusetts  148         Boston   
10  2622000                 Detroit city, Michigan  220        Detroit   
11  2743000            Minneapolis city, Minnesota  378    Minneapolis   
12  2758000               St. Paul city, Minnesota  378       St. Paul   
13  4260000        Philadelphia city, Pennsylvania  428   Philadelphia   
14  4819000                     Dallas city, Texas  206         Dallas   
15  4827000                 Fort Worth city, Texas  206     Fort Worth   
16  4835000                    Houston city, Texas  288        Houston   
17  5363000               Seattle city, Washington  500        Seattle   
18  3651000                New York city, New York  408       New York   

    1-2units  +3units  
0      16413    73435  
1       1061     8600  
2        890    26362  
3       1862    22477  
4      14679    36980  
5       3196    30767  
6        641    18306  
7       4965    31340  
8       4442    43092  
9       1097    21450  
10       704     2878  
11       987    16502  
12       452     3500  
13      7915    16462  
14     12489    59802  
15     33656    22621  
16     40686    66918  
17      8042    56102  
18     13409   185543

### Exporting all places' housing permits 2000-2017 data to .csv

In [18]:
BPS_place.to_csv('exports/BP00-08-17places.csv')